In [1]:
import pandas as pd
import numpy as np

In [4]:
halan_regression3 = pd.read_csv('dash_application/csv/weekly_data2.csv')
halan_regression3['average']=halan_regression3.iloc[:,1:5].mean(axis=1)
df = halan_regression3.copy()
df['w1']=df['w1']/df['average']
df['w2']=df['w2']/df['average']
df['w3']=df['w3']/df['average']
df['w4']=df['w4']/df['average']

columns = ['w1','w2','w3','w4']
deseasonalized = pd.DataFrame()

deseasonalized['Month']=[halan_regression3['Month'][j] for j in range(len(df)) for i in columns]
deseasonalized['Week']=[int(i[-1]) for j in range(len(df)) for i in columns]


deseasonalized['Original']=[halan_regression3[i][j] for j in range(len(df)) for i in columns]
deseasonalized['Delivered'] =[halan_regression3[i][j]/df[i].mean(axis=0) for j in range(len(df)) for i in columns]       
deseasonalized['MA12'] = deseasonalized['Original'].rolling(5).mean()# plot the data and MA
deseasonalized['Series'] = np.arange(1,len(deseasonalized)+1)



In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso

# evaluate an Passive agressive regression model on the dataset
from numpy import mean
from numpy import std
from numpy import absolute

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold



# drop unnecessary columns and re-arrange
data = deseasonalized[['Series', 'Month', 'Week', 'Original']]
x = np.array(data.drop(["Series","Original"], 1))
y = np.array(data["Original"])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.1, random_state=42)
model = Lasso(alpha=1.0, copy_X=True,
                fit_intercept=True,
                max_iter=1000, normalize=False,
                positive=False,
                precompute=False,
                random_state=123,
                selection='cyclic', tol=0.0001,
                warm_start=False)
model.fit(xtrain, ytrain)
ypred = model.predict(xtest)

pred_del = pd.DataFrame(data={"Predicted Delivered": ypred.flatten()})
print(pred_del.head())

cv = RepeatedKFold(n_splits=8, n_repeats=30, random_state=1)
# define model

# fit model


# evaluate model
scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_31380\3714764680.py:18: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

c:\Users\moham\OneDrive - University of Nottingham Malaysia Campus\Future University\Halan_Dashboard\project_env\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.

c:\Users\moham\OneDrive - University of Nottingham Malaysia Campus\Future University\Halan_Dashboard\project_env\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to

   Predicted Delivered
0           481.268212
1           494.281002
2           468.255423


c:\Users\moham\OneDrive - University of Nottingham Malaysia Campus\Future University\Halan_Dashboard\project_env\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.

c:\Users\moham\OneDrive - University of Nottingham Malaysia Campus\Future University\Halan_Dashboard\project_env\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.

c:\Users\moham\One

Mean MAE: 100.443 (48.829)


c:\Users\moham\OneDrive - University of Nottingham Malaysia Campus\Future University\Halan_Dashboard\project_env\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.

c:\Users\moham\OneDrive - University of Nottingham Malaysia Campus\Future University\Halan_Dashboard\project_env\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.

c:\Users\moham\One

In [84]:

future_df = pd.DataFrame()
future_df['Month'] =  [i for i in  np.arange(1,13) for j in  np.arange(1,5)]   
future_df['Week'] = [j for i in  np.arange(1,13) for j in  np.arange(1,5)]  
future_df['Series'] = np.arange(0,(48))

predictions_future = model.predict( future_df.drop(['Series'],axis=1,))
predictions = pd.DataFrame(predictions_future, columns=['Label'])
future_df['Label']=predictions['Label']

concat_df = pd.concat([data,future_df], axis=0)


fig = px.line(concat_df, x="Series", y=["Original", "Label"], template = 'plotly_dark')
fig.show()

c:\Users\moham\OneDrive - University of Nottingham Malaysia Campus\Future University\Halan_Dashboard\project_env\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but Lasso was fitted without feature names



In [85]:
df2 = df.rename({'w1': 1, 'w2': 2,'w3':3,'w4':4}, axis=1) 
avgs=[]
for i in range(4):
  avgs.append(df2[i+1].mean())
res = concat_df.dropna(subset=['Label'], axis=0)

res.drop(['Original'], axis=1, inplace=True)
res['reseason']= res['Label']   
bias = 2
hold= 1
for i in range(len(res['reseason'])):
  res['reseason'][i]=res['reseason'][i]*avgs[res['Week'][i]-hold]+bias*i
concat_res = pd.concat([res,deseasonalized], axis=0)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_31380\4037807315.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_31380\4037807315.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_31380\4037807315.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [86]:

fig = px.line(concat_res, x="Series", y=["reseason",'Label',"Original"], template = 'plotly_dark')
fig.show()